# Accessing remote files with earthaccess 

When we search for data using earthaccess we get back a list of results from NASA's Common Metadata Repository or CMR for short. These results contain all the information
we need to access the files represented by the metadata. `earthaccess` offers 2 access methods that operate with these results, the first method is the well known, `download()`
where we copy the results from their location to our local disk, if we are running the code in AWS say on a Jupyterhub the files will be copied to the local VM disk. 
The other method is `open()`, earthaccess uses fsspec to open remote files as if they were local. `open` has advantages and some disadvantages that we must know before using it. 

The main advantage for `open()` is that we don't have to download the file, we can stream it into memory however depending on how we do it we may run into network performance issues. Again, if we run the code next to the data this would be fast, if we do it locally in our laptopts it will be slow. 

In [ ]:
import earthaccess

auth = earthaccess.login()

In [ ]:
results = earthaccess.search_data(
    short_name="ATL06",
    cloud_hosted=False,
    temporal=("2019-01", "2019-02"),
    polygon=[(-100, 40), (-110, 40), (-105, 38), (-100, 40)],
)
results[0]

In [ ]:
nsidc_url = "https://n5eil01u.ecs.nsidc.org/DP7/ATLAS/ATL06.005/2019.02.21/ATL06_20190221121851_08410203_005_01.h5"
lpcloud_url = "https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/EMITL2ARFL.001/EMIT_L2A_RFL_001_20220903T163129_2224611_012/EMIT_L2A_RFL_001_20220903T163129_2224611_012.nc"

session = earthaccess.get_requests_https_session()

In [ ]:
headers = {"Range": "bytes=0-100"}
r = session.get(lpcloud_url, headers=headers)
r

In [ ]:
fs = earthaccess.get_fsspec_https_session()

In [ ]:
with fs.open(lpcloud_url) as f:
    data = f.read(100)
data

In [ ]:
%%time

import xarray as xr

files = earthaccess.open(results[0:2])

ds = xr.open_dataset(files[0], group="/gt1r/land_ice_segments")
ds